# Prédiction de l'Attrition Client Bancaire : Pipeline de Machine Learning Distribué avec Apache Spark

* imports : 

In [24]:
from pyspark.sql import SparkSession


## 🟢 Configuration et Initialisation de Spark

### 1️⃣ Configuration de l’environnement Spark

#### - Installer et configurer PySpark

In [25]:
# Installation des dépendances ( exécuter dans le terminal)
"""
pip install pyspark==3.5.0
pip install pymongo
pip install findspark
"""

'\npip install pyspark==3.5.0\npip install pymongo\npip install findspark\n'

#### - Créer et tester une SparkSession

In [26]:

spark = SparkSession.builder \
    .appName("Banking_Churn_Prediction") \
    .master("local[*]") \
    .getOrCreate()




#### - Vérifier la version et les paramètres d’exécution

In [27]:
print("✅ SparkSession initialisée avec succès !")
print(f"Version de Spark : {spark.version}")

✅ SparkSession initialisée avec succès !
Version de Spark : 4.0.1


In [28]:
# Afficher quelques configurations importantes
print("\n⚙️  Paramètres Spark en cours d'exécution :")
for key, value in spark.sparkContext.getConf().getAll():
    print(f"{key} = {value}")



⚙️  Paramètres Spark en cours d'exécution :
spark.rdd.compress = True
spark.hadoop.fs.s3a.vectored.read.min.seek.size = 128K
spark.app.startTime = 1762191534888
spark.executor.extraJavaOptions = -Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-modules=jdk.incubator.vector --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java

### 2️⃣ Chargement des Données

#### - Charger les données brutes au format CSV dans un DataFrame Spark avec : spark.read.csv

In [29]:
df_clients = spark.read.csv(
    "../data/dataset.csv",
    header=True,
    inferSchema=True,
    sep=",",
    nullValue="NA"
)

print(f"✅ Données chargées avec succés !")
print(f"Nombre de lignes ; {df_clients.count()}")
print(f"Nombre de colonnes : {len(df_clients.columns)}")

✅ Données chargées avec succés !
Nombre de lignes ; 10000
Nombre de colonnes : 14


In [30]:
df_clients

DataFrame[RowNumber: int, CustomerId: int, Surname: string, CreditScore: int, Geography: string, Gender: string, Age: int, Tenure: int, Balance: double, NumOfProducts: int, HasCrCard: int, IsActiveMember: int, EstimatedSalary: double, Exited: int]

#### - Afficher le schéma des colonnes 

In [31]:
print(f"Schéma des colonnes : ")
df_clients.printSchema()

Schéma des colonnes : 
root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



#### - Afficher un aperçu des premières lignes


In [32]:
print("Aperçu des données :")
df_clients.show(5, truncate=False)

Aperçu des données :
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|Surname |CreditScore|Geography|Gender|Age|Tenure|Balance  |NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|1        |15634602  |Hargrave|619        |France   |Female|42 |2     |0.0      |1            |1        |1             |101348.88      |1     |
|2        |15647311  |Hill    |608        |Spain    |Female|41 |1     |83807.86 |1            |0        |1             |112542.58      |0     |
|3        |15619304  |Onio    |502        |France   |Female|42 |8     |159660.8 |3            |1        |0             |113931.57      |1     |
|4        |15701354  |Boni    |699        |France   |Female|39 |1     |0.0      |2            |0        |0         

* stoper spark : 

In [33]:
spark.stop()